# Open Weather API updated

- This Python notebook is an updated version of open_weather_api.ipynb.
The code has been made significantly more robust (less error-prone) and all functionality has been thoroughly reviewed and tested again.
This file is more cleand and organized then the last one

In [1]:
import requests
import os
from datetime import datetime, timezone, timedelta
import pandas as pd

In [2]:
try:
    api_key = os.getenv("weather_api")
except KeyError:
    raise RuntimeError("API key not set. Please set OPENWEATHER_API_KEY.")

base_url = "https://api.openweathermap.org/data/2.5"

In [3]:
# Function to fetch real time weather data for any named city or any specific location(using geographical coordinates)
def get_current_data(city=None, lat=None, lon=None):
    """ get current weather location of any location or city
    Args:
        1)city (str) : City name or
        2)lat (int): latitude
        3)lon (int) longitude
    **Either provide city name or lat and lon
    Returns:
        A python dictionary
    """

    if city:
        params = {"q": city, "appid": api_key, "units": "metric"}
    elif lat is not None and lon is not None:
        params = {"lat": lat, "lon": lon, "appid": api_key, "units": "metric"}
    else:
        # print("Either provide a city name or latitude & longitude values.")
        return {"success": False, "error": "No location provided. Please enter a city or latitude & longitude.", "data": None }

    
    try:
        response = requests.get(f"{base_url}/weather", params=params , timeout=10)
        response.raise_for_status()
        return {"success": True, "data": _filter_current_data(response.json()), "error": None}

    except requests.exceptions.Timeout:
        return {"success": False, "error": "Error: Request timed out, Try again later.", "data": None}

    except requests.exceptions.ConnectionError:
        return {"success": False, "error": "Error: Network problem, Check your internet connection", "data": None}

    except requests.exceptions.HTTPError:
        return {"success": False, "error": "Error: Invalid city name or coordinates", "data": None}

    except Exception as e:
        return {"success": False, "error": f"Unexpected error: {e}", "data": None}

In [4]:
# this function extracts only useful information from the row JSON file and stores in a python dictionary.

def _filter_current_data(data):
    """
    Extracts only useful information from the raw JSON file and returns a well-formatted Python dictionary.
    Args:
        data (dict): current weather JSON response from API
    Returns:
        dict: dictionary with cleaned and safe weather information
    """

    tz = timezone(timedelta(seconds=data.get("timezone", 0)))
    def to_local_time(timestamp):
        if timestamp is None:
            return None
        try:
            return datetime.fromtimestamp(timestamp, tz=timezone.utc).astimezone(tz).strftime("%Y-%m-%d %H:%M:%S")
        except Exception:
            return None

    main = data.get("main", {})
    sys_data = data.get("sys", {})

    mydict = {
        "city": data.get("name", "-"),
        "date_time": to_local_time(data.get("dt")),
        "temperature": main.get("temp"),
        "feels_like": main.get("feels_like"),
        "weather_description": data.get("weather", [{}])[0].get("description", "-"),
        "pressure": main.get("pressure"),
        "humidity": main.get("humidity"),
        "visibility": data.get("visibility"),
        "wind_speed": data.get("wind", {}).get("speed"),
        "sunrise": to_local_time(sys_data.get("sunrise")),
        "sunset": to_local_time(sys_data.get("sunset")),

        "long": data.get("coord", {}).get("lon"),
        "lat": data.get("coord", {}).get("lat"),
        "timezone": data.get("timezone", 0),
        "country": sys_data.get("country", "-"),
    }

    return mydict


In [5]:
# Fetching data using city name
get_current_data(city="Tokyo")

{'success': True,
 'data': {'city': 'Tokyo',
  'date_time': '2025-12-27 23:21:49',
  'temperature': 4.52,
  'feels_like': 2.27,
  'weather_description': 'few clouds',
  'pressure': 1019,
  'humidity': 58,
  'visibility': 10000,
  'wind_speed': 2.57,
  'sunrise': '2025-12-27 06:49:34',
  'sunset': '2025-12-27 16:34:29',
  'long': 139.6917,
  'lat': 35.6895,
  'timezone': 32400,
  'country': 'JP'},
 'error': None}

In [6]:
# Fetching data using coordinates
get_current_data(lat=35.6895, lon=139.6917)

{'success': True,
 'data': {'city': 'Tokyo',
  'date_time': '2025-12-27 23:20:28',
  'temperature': 4.52,
  'feels_like': 2.27,
  'weather_description': 'few clouds',
  'pressure': 1019,
  'humidity': 58,
  'visibility': 10000,
  'wind_speed': 2.57,
  'sunrise': '2025-12-27 06:49:34',
  'sunset': '2025-12-27 16:34:29',
  'long': 139.6917,
  'lat': 35.6895,
  'timezone': 32400,
  'country': 'JP'},
 'error': None}

In [7]:
def get_multiple_city(cities):
    """ Fetch weather data for multiple cities.
    Args:
        cities (list of str): List of city names.
    Returns:
        list of dict: Each dict contains processed weather info."""

    results = []
    for city in cities:
        res = get_current_data(city=city)

        results.append({
            "city": city,
            "success": res["success"],
            "data": res["data"],
            "error": res["error"]})

    return results

In [8]:
data = get_multiple_city(["Tokyo", "New delhii", "New York"])

for item in data:
    for key, value in item.items():
        print(key)
        print("\t", value)
    print("\n")        

city
	 Tokyo
success
	 True
data
	 {'city': 'Tokyo', 'date_time': '2025-12-27 23:21:49', 'temperature': 4.52, 'feels_like': 2.27, 'weather_description': 'few clouds', 'pressure': 1019, 'humidity': 58, 'visibility': 10000, 'wind_speed': 2.57, 'sunrise': '2025-12-27 06:49:34', 'sunset': '2025-12-27 16:34:29', 'long': 139.6917, 'lat': 35.6895, 'timezone': 32400, 'country': 'JP'}
error
	 None


city
	 New delhii
success
	 False
data
	 None
error
	 Error: Invalid city name or coordinates


city
	 New York
success
	 True
data
	 {'city': 'New York', 'date_time': '2025-12-27 09:22:54', 'temperature': -3.73, 'feels_like': -10.73, 'weather_description': 'light snow', 'pressure': 1018, 'humidity': 88, 'visibility': 1207, 'wind_speed': 7.6, 'sunrise': '2025-12-27 07:19:03', 'sunset': '2025-12-27 16:35:09', 'long': -74.006, 'lat': 40.7143, 'timezone': -18000, 'country': 'US'}
error
	 None




In [9]:
def get_multiple_location(coordinates):
    """
    Fetch weather data for multiple coordinates.
    Args:
        coordinates (list of tuples): List of (lat, lon) tuples.
    Returns:
        list of dict: Each dict contains processed weather info.
    """
    results = []
    for lat, lon in coordinates:
        res = get_current_data(lat=lat, lon=lon)

        results.append({
            "lat": lat,
            "lon": lon,
            "success": res["success"],
            "data": res["data"],
            "error": res["error"]})

    return results

In [10]:
data = get_multiple_location([(40.7143,-74.006), (35.6895,139.6917), ("a",-74.006)])

for item in data:
    for key, value in item.items():
        print(key)
        print("\t", value)
    print("\n")        

lat
	 40.7143
lon
	 -74.006
success
	 True
data
	 {'city': 'New York', 'date_time': '2025-12-27 09:15:59', 'temperature': -3.76, 'feels_like': -10.76, 'weather_description': 'light snow', 'pressure': 1018, 'humidity': 88, 'visibility': 1207, 'wind_speed': 7.6, 'sunrise': '2025-12-27 07:19:03', 'sunset': '2025-12-27 16:35:09', 'long': -74.006, 'lat': 40.7127, 'timezone': -18000, 'country': 'US'}
error
	 None


lat
	 35.6895
lon
	 139.6917
success
	 True
data
	 {'city': 'Tokyo', 'date_time': '2025-12-27 23:20:28', 'temperature': 4.52, 'feels_like': 2.27, 'weather_description': 'few clouds', 'pressure': 1019, 'humidity': 58, 'visibility': 10000, 'wind_speed': 2.57, 'sunrise': '2025-12-27 06:49:34', 'sunset': '2025-12-27 16:34:29', 'long': 139.6917, 'lat': 35.6895, 'timezone': 32400, 'country': 'JP'}
error
	 None


lat
	 a
lon
	 -74.006
success
	 False
data
	 None
error
	 Error: Invalid city name or coordinates




In [11]:
# This function fetch 5 days forecast day with 3-hour step for any location.
def get_forecast_data(city=None, lat=None, lon=None):
    """ get forecast data of any location or city
    Args:
        1)city (str) : City name  or
        2)lat (int): latitude
        3)lon (int) longitude
    **Either provide city name or lat and lon
    Returns:
        A JSON file
    """

    if city:
        params = {"q": city, "appid": api_key, "units": "metric"}

    elif lat is not None and lon is not None:
        # url = f"{base_url}/forecast?lat={lat}&lon={lon}&appid={api_key}&units=metric"
        params = {"lat":lat, "lon": lon, "appid": api_key, "units": "metric"}
    else:
        return {"success": False, "error": "No location provided. Please enter a city or latitude & longitude.", "data": None}

    
    try:
        response = requests.get(f"{base_url}/forecast", params=params, timeout=10)
        response.raise_for_status()
        return {"success": True, "data": filter_forecast_data(response.json()), "error": None}

    except requests.exceptions.RequestException as e:
        return {"success": False, "error": "Unable to fetch weather data at the moment. Please try again later.", "data": None}

In [12]:
def filter_forecast_data(data):
    """ filter forecast data and return useful information
    Args:
        data(JSON): forecast data
    Returns:
        pyhton list:the python list contains two elements
                1) the metadata dictionary, which stores location into
                2) the weather list, which contains forecast data for the location
    """

    # metadata
    city_info = {
        "city": data.get("city", {}).get("name", "-"),
        "latitude": data.get("city",{}).get("coord", {}).get("lat"),
        "longitude": data.get("city",{}).get("coord", {}).get("lon"),
        "country": data.get("city", {}).get("country", "-"),
        "timezone": data.get("city", {}).get("timezone", 0)
    }

    infolist = []
    forecast_list = data.get("list", [])

    for item in forecast_list:
        main = item.get("main", {})
        weather_list = item.get("weather", [{}])

        
        mydict = {"date_str": item.get("dt_txt", "-"),
                  "temperature": main.get("temp"),
                  "feels_like": main.get("feels_like"),
                  "weather_description": weather_list[0].get("description", "-") ,
                  "pressure": main.get("pressure"),
                  "humidity": main.get("humidity"),
                  "visibility": item.get("visibility"),
                  "wind_speed": main.get("wind", {}).get("speed"),
                  "date_time": item.get("dt")
                   }

        infolist.append(mydict)
    return {
        "city": city_info,
        "forecast": infolist
    }


In [13]:
data = get_forecast_data("Mumbai")

In [14]:
data["data"]["city"]

{'city': 'Mumbai',
 'latitude': 19.0144,
 'longitude': 72.8479,
 'country': 'IN',
 'timezone': 19800}

In [15]:
df = pd.DataFrame(data["data"]["forecast"])
df.head()

,date_str,temperature,feels_like,weather_description,pressure,humidity,visibility,wind_speed,date_time
0,2025-12-27 15:00:00,26.99,27.66,clear sky,1013,54,10000,None,1766847600
1,2025-12-27 18:00:00,26.35,26.35,clear sky,1013,55,10000,None,1766858400
2,2025-12-27 21:00:00,25.29,25.26,clear sky,1012,53,10000,None,1766869200
3,2025-12-28 00:00:00,23.69,23.47,clear sky,1012,52,10000,None,1766880000
4,2025-12-28 03:00:00,23.30,23.02,clear sky,1015,51,10000,None,1766890800


In [16]:
data = get_forecast_data(lat=28.6128, lon=77.2311)
df = pd.DataFrame(data["data"]["forecast"])
df.head()

,date_str,temperature,feels_like,weather_description,pressure,humidity,visibility,wind_speed,date_time
0,2025-12-27 15:00:00,16.09,15.50,scattered clouds,1016,67,10000,None,1766847600
1,2025-12-27 18:00:00,16.52,15.59,scattered clouds,1016,52,10000,None,1766858400
2,2025-12-27 21:00:00,16.16,14.82,scattered clouds,1016,38,10000,None,1766869200
3,2025-12-28 00:00:00,15.02,13.23,scattered clouds,1016,25,10000,None,1766880000
4,2025-12-28 03:00:00,15.22,13.40,clear sky,1018,23,10000,None,1766890800


In [17]:
data["data"]["city"]

{'city': 'New Delhi',
 'latitude': 28.6128,
 'longitude': 77.2311,
 'country': 'IN',
 'timezone': 19800}